In [1]:
#-----------------------
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, concatenate
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Dense, Activation, Lambda, Flatten
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.models import load_model
from keras.models import model_from_json
from keras.layers.merge import Concatenate
from keras import backend as K
from model import create_model #research-papers/facenet
#from os import listdir
import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import time
#-----------------------
face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt","res10_300x300_ssd_iter_140000.caffemodel")

In [2]:
model = create_model()
print("Facenet model built.")
model.load_weights('nn4.small2.v1.h5') #research-papers/facenet
print("Facenet weights built.")

Facenet model built.
Facenet weights built.


In [3]:
def preprocess_image(img):
    if type(img) == type('str'):
        img = cv2.imread(img)
        img = ssd_getDetectFace(img)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convet grayscale
        img = cv2.resize(img, (96, 96))
        img = img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        #img = preprocess_input(img) ## [-1, 1]
        img /= 255 # [0, 1]
    else:
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convet grayscale
        img = cv2.resize(img, (96, 96))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        #img = preprocess_input(img) ## [-1, 1]
        img /= 255  # [0, 1]
    return img

In [4]:
#distance
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [5]:
def ssd_detectFace(img):
    
    target_size = (300, 300)
    
    img = cv2.resize(img, target_size)

    imageBlob = cv2.dnn.blobFromImage(img)

    face_detector.setInput(imageBlob)
    detections = face_detector.forward()
    
    return detections

In [6]:
def ssd_getDetectFace(img):
    ssd_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]

    target_size = (300, 300)

    base_img = img.copy() #we will restore base_img to img later

    original_size = img.shape

    img = cv2.resize(img, target_size)

    aspect_ratio_x = (original_size[1] / target_size[1])
    aspect_ratio_y = (original_size[0] / target_size[0])

    imageBlob = cv2.dnn.blobFromImage(image = img)
    
    face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt","res10_300x300_ssd_iter_140000.caffemodel")
    face_detector.setInput(imageBlob)
    detections = face_detector.forward()

    detections_df = pd.DataFrame(detections[0][0], columns = ssd_labels)

    detections_df = detections_df[detections_df['is_face'] == 1] #0: background, 1: face
    detections_df = detections_df[detections_df['confidence'] >= 0.90]

    detections_df['left'] = (detections_df['left'] * 300).astype(int)
    detections_df['bottom'] = (detections_df['bottom'] * 300).astype(int)
    detections_df['right'] = (detections_df['right'] * 300).astype(int)
    detections_df['top'] = (detections_df['top'] * 300).astype(int)
    
    if detections_df.shape[0] > 0:

        #TODO: sort detections_df

        #get the first face in the image
        instance = detections_df.iloc[0]

        left = int(instance["left"] * aspect_ratio_x)
        right = int(instance["right"] * aspect_ratio_x)
        bottom = int(instance["bottom"] * aspect_ratio_y)
        top = int(instance["top"] * aspect_ratio_y)

        detected_face = base_img[top:bottom, left:right]
        
        #cv2.rectangle(img, (left, top), (right, bottom),(10, 10, 255), 3)
        
        return detected_face[:,:,::-1] #convert RGB

In [15]:
employees = []

db_path = '.\database'

for r, d, f in os.walk(db_path): # r=root, d=directories, f = files
    for file in f:
        print(file)
        img = preprocess_image(r + "\\" + file)
        
        representation = model.predict(img)[0,:]
        name = r.replace(db_path,"").replace("\\","")
        
        employees.append([representation,name])

print("employee representations retrieved successfully")

LINE_ALBUM_2023831_230831_1.jpg
LINE_ALBUM_2023831_230831_10.jpg
LINE_ALBUM_2023831_230831_11.jpg
LINE_ALBUM_2023831_230831_12.jpg
LINE_ALBUM_2023831_230831_13.jpg
LINE_ALBUM_2023831_230831_14.jpg
LINE_ALBUM_2023831_230831_15.jpg
LINE_ALBUM_2023831_230831_2.jpg
LINE_ALBUM_2023831_230831_3.jpg
LINE_ALBUM_2023831_230831_4.jpg
LINE_ALBUM_2023831_230831_5.jpg
LINE_ALBUM_2023831_230831_6.jpg
LINE_ALBUM_2023831_230831_7.jpg
LINE_ALBUM_2023831_230831_8.jpg
LINE_ALBUM_2023831_230831_9.jpg
977.png
d8a24b317b1d544cf1fefc16628ef0a7.png
gettyimages-139369178-594x594.jpg
gettyimages-490703338.jpg
gettyimages-86146687-612x612.jpg
gettyimages-93243663-612x612.jpg
i.png
Kobe_Bryant_2014.jpg
Kobe_Bryant_2015.jpg
phpflly88.jpg
16752529896605.jpg
16852041551656.jpg
2544.png
504c63a03d8a751a5cbeda0bc064306bb4-lebron-james.rsquare.w700.png
c05041402158dc67fcd77e5512d0bed5.png
i_8e_7f_1e_lebron-james.png
LeBron-1040x572.jpg
lebron-james-b73c66eb3444477c8896a44a3e39b671.jpg
LeBron_James_(51959977144)_(croppe

In [16]:
cap = cv2.VideoCapture(0) #webcam
counter = 0
start_time = time.time()

In [ ]:
while True:
    
    ret, img = cap.read()
    img = cv2.flip(img, 1)

    (h, w) = np.array(img).shape[:2]

    detections = ssd_detectFace(img)

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence < 0.9:
            continue

        box = detections[0, 0, i, 3:] * np.array([w, h, w, h])
        (left, top, right, bottom) = box.astype("int") #L/T/R/B

        label_name = 'unknown'
        threshold = 0.55

        detected_face = img[top:bottom,left:right][:,:,::-1] #convert RGB
        preprocess_img = preprocess_image(detected_face)
        captured_representation = model.predict(preprocess_img)[0,:]
        
        for employee in employees:
            source_representation = employee[0]
            
            distance = findEuclideanDistance(captured_representation, source_representation)

            if distance <= threshold:
                label_name = employee[1]
                threshold = distance
                
        if label_name == 'unknown':
            text = label_name
        else:
            #threshold = 0.65 -> 75
            #threshold = 0.0 -> 100
            score = threshold*-400/11+100 
            text = label_name + " {:.2f}%".format(score)
        
        color = (50, 50, 200) # R(50, 50, 200) G(50, 200, 50) B(200,50,50) Y(255,200,50) B(0,200,255)
        cv2.rectangle(img, (left, top), (right, bottom), color, 3)
        cv2.rectangle(img, (left, top), (right, top+25), color,-1)
        
        #FONT_HERSHEY_SIMPLEX or FONT_HERSHEY_DUPLEX
        
        if(right-left < 100):
            
            cv2.putText(img, text, (left, top+15),cv2.FONT_HERSHEY_DUPLEX, 0.25, (255, 255, 255), 1,cv2.LINE_AA)
        elif(right-left < 120):
            cv2.putText(img, text, (left, top+15),cv2.FONT_HERSHEY_DUPLEX, 0.30, (255, 255, 255), 1,cv2.LINE_AA)
        elif(right-left < 150):
            cv2.putText(img, text, (left, top+15),cv2.FONT_HERSHEY_DUPLEX, 0.35, (255, 255, 255), 1,cv2.LINE_AA)
        else:
            cv2.putText(img, text, (left, top+15),cv2.FONT_HERSHEY_DUPLEX, 0.45, (255, 255, 255), 1,cv2.LINE_AA)
        
        counter += 1
        if (time.time() - start_time) != 0:
            fps = counter / (time.time() - start_time)
            #cv2.rectangle(img, (0, 0), (60, 25),(50, 50, 50),-1)
            cv2.putText(img, "FPS:{:.0f}".format(fps), (2, 15),cv2.FONT_HERSHEY_DUPLEX, 0.45, (100, 100, 100), 1,cv2.LINE_AA)

        counter = 0
        start_time = time.time()

    cv2.imshow("SSD Face Detection and OpenFace Face Recognition", img)

    if cv2.waitKey(1) & 0xFF == ord("q"): #press q to quit
        break

#kill open cvthings
cap.release()
cv2.destroyAllWindows()